In [452]:
import random
import json
import numpy as np
from datetime import datetime
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import pymorphy2
morph = pymorphy2.MorphAnalyzer()
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\artzh\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\artzh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [453]:
name = "Нейрон!"

In [454]:
lemmatizer = nltk.WordNetLemmatizer()

intents = json.loads(open("intents.json", encoding='utf-8').read())

print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Привет', 'Здравствуй', 'Хай', 'Ку', 'Добрый день', 'Приветствую'], 'responses': ['Привет', 'Аллоха', 'Здравствуй, Незнакомец', 'Как я могут помочь тебе']}, {'tag': 'goodbye', 'patterns': ['Пока', 'До свидания', 'Давай', 'бб', 'До встречи', 'Всего доброго'], 'responses': ['До встречи', 'Увидимся!', 'Всего доброго', 'Бывай']}, {'tag': 'howfiilig', 'patterns': ['Как ты?', 'Как дела?', 'Как настроение?'], 'responses': ['Хорошо, а ты как?', 'Неплохо', 'Нормально, сегодня система сбоит']}, {'tag': 'age', 'patterns': ['Сколько тебе лет?', 'Сколько тебе, Нейрон?', 'Возраст?'], 'responses': ['Мне 18', 'Мне всего 18']}, {'tag': 'name', 'patterns': ['Как тебя зовут?', 'Как к Вам обращаться?', 'Кто ты?'], 'responses': ['Ты можешь называть меня *name', 'Я *name']}, {'tag': 'time', 'patterns': ['Сколько сейчас времени?', 'Который час?'], 'responses': ["Сейчас *datetime.now().strftime('%X')"]}, {'tag': 'date', 'patterns': ['Какой сегодня день?', 'Какое с

In [455]:
words = []
clasees = []
documents = []
ignore_letters = ['?', '!', '.', ',']
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Привет', 'Здравствуй', 'Хай', 'Ку', 'Добрый день', 'Приветствую'], 'responses': ['Привет', 'Аллоха', 'Здравствуй, Незнакомец', 'Как я могут помочь тебе']}, {'tag': 'goodbye', 'patterns': ['Пока', 'До свидания', 'Давай', 'бб', 'До встречи', 'Всего доброго'], 'responses': ['До встречи', 'Увидимся!', 'Всего доброго', 'Бывай']}, {'tag': 'howfiilig', 'patterns': ['Как ты?', 'Как дела?', 'Как настроение?'], 'responses': ['Хорошо, а ты как?', 'Неплохо', 'Нормально, сегодня система сбоит']}, {'tag': 'age', 'patterns': ['Сколько тебе лет?', 'Сколько тебе, Нейрон?', 'Возраст?'], 'responses': ['Мне 18', 'Мне всего 18']}, {'tag': 'name', 'patterns': ['Как тебя зовут?', 'Как к Вам обращаться?', 'Кто ты?'], 'responses': ['Ты можешь называть меня *name', 'Я *name']}, {'tag': 'time', 'patterns': ['Сколько сейчас времени?', 'Который час?'], 'responses': ["Сейчас *datetime.now().strftime('%X')"]}, {'tag': 'date', 'patterns': ['Какой сегодня день?', 'Какое с

In [456]:
for intent in intents['intents']:
    for pattern in intent['patterns']:
        word_list = nltk.word_tokenize(pattern)
        words.append(word_list)
        documents.append(((word_list), intent["tag"]))
        if intent['tag'] not in clasees:
            clasees.append(intent['tag'])
           

In [457]:
print(intents)

{'intents': [{'tag': 'greeting', 'patterns': ['Привет', 'Здравствуй', 'Хай', 'Ку', 'Добрый день', 'Приветствую'], 'responses': ['Привет', 'Аллоха', 'Здравствуй, Незнакомец', 'Как я могут помочь тебе']}, {'tag': 'goodbye', 'patterns': ['Пока', 'До свидания', 'Давай', 'бб', 'До встречи', 'Всего доброго'], 'responses': ['До встречи', 'Увидимся!', 'Всего доброго', 'Бывай']}, {'tag': 'howfiilig', 'patterns': ['Как ты?', 'Как дела?', 'Как настроение?'], 'responses': ['Хорошо, а ты как?', 'Неплохо', 'Нормально, сегодня система сбоит']}, {'tag': 'age', 'patterns': ['Сколько тебе лет?', 'Сколько тебе, Нейрон?', 'Возраст?'], 'responses': ['Мне 18', 'Мне всего 18']}, {'tag': 'name', 'patterns': ['Как тебя зовут?', 'Как к Вам обращаться?', 'Кто ты?'], 'responses': ['Ты можешь называть меня *name', 'Я *name']}, {'tag': 'time', 'patterns': ['Сколько сейчас времени?', 'Который час?'], 'responses': ["Сейчас *datetime.now().strftime('%X')"]}, {'tag': 'date', 'patterns': ['Какой сегодня день?', 'Какое с

In [458]:
new_words = []
for secuence in words:
        for word in secuence:
            new_words.append(morph.parse(word)[0])     
words = [word.normal_form for word in new_words if word not in ignore_letters] # лематизируем все слова не входящие в список игноров
words = sorted(set(words))
clasees = sorted(set(clasees))
print(words)

[',', '?', 'бб', 'весь', 'возраст', 'время', 'встреча', 'вы', 'год', 'давать', 'дело', 'день', 'до', 'добрый', 'звать', 'здравствуй', 'к', 'как', 'какой', 'который', 'кто', 'ку', 'настроение', 'нейрон', 'обращаться', 'пока', 'привет', 'приветствовать', 'свидание', 'сегодня', 'сейчас', 'сколько', 'ты', 'хай', 'час', 'число']


In [459]:
training = []
output_empty = [0] * len(clasees)

In [460]:
for document in documents:
    bag = []
    word_pattern = document[0]
    word_pattern = [morph.parse(word.lower())[0].normal_form for word in word_pattern]
    for word in words:
        bag.append(1) if word in word_pattern else bag.append(0)
    output_row = list(output_empty)
    output_row[clasees.index(document[1])] = 1
    training.append([bag, output_row])


In [461]:
print(words)

[',', '?', 'бб', 'весь', 'возраст', 'время', 'встреча', 'вы', 'год', 'давать', 'дело', 'день', 'до', 'добрый', 'звать', 'здравствуй', 'к', 'как', 'какой', 'который', 'кто', 'ку', 'настроение', 'нейрон', 'обращаться', 'пока', 'привет', 'приветствовать', 'свидание', 'сегодня', 'сейчас', 'сколько', 'ты', 'хай', 'час', 'число']


In [462]:
print(training)

[[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0], [0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 1, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 1, 0, 0, 0, 0]], [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0

In [463]:
random.shuffle(training)
training = np.array(training)

C:\Users\artzh\AppData\Local\Temp/ipykernel_21772/2195589081.py:2: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  training = np.array(training)


In [464]:
train_x = list(training[:, 0])
train_y = list(training[:, 1])

In [465]:
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

In [466]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)

In [467]:
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [468]:
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)

Epoch 1/200
5/5 [==============================] - 0s 1ms/step - loss: 1.9306 - accuracy: 0.2800
Epoch 2/200
5/5 [==============================] - 0s 1ms/step - loss: 1.9213 - accuracy: 0.1600
Epoch 3/200
5/5 [==============================] - 0s 1ms/step - loss: 1.8986 - accuracy: 0.2000
Epoch 4/200
5/5 [==============================] - 0s 1ms/step - loss: 1.7808 - accuracy: 0.4400
Epoch 5/200
5/5 [==============================] - 0s 998us/step - loss: 1.7792 - accuracy: 0.3200
Epoch 6/200
5/5 [==============================] - 0s 1ms/step - loss: 1.8120 - accuracy: 0.2800
Epoch 7/200
5/5 [==============================] - 0s 997us/step - loss: 1.6529 - accuracy: 0.4800
Epoch 8/200
5/5 [==============================] - 0s 1ms/step - loss: 1.5408 - accuracy: 0.5600
Epoch 9/200
5/5 [==============================] - 0s 998us/step - loss: 1.5678 - accuracy: 0.4800
Epoch 10/200
5/5 [==============================] - 0s 1ms/step - loss: 1.5226 - accuracy: 0.4800
Epoch 11/200
5/5 [=====

5/5 [==============================] - 0s 1ms/step - loss: 0.1104 - accuracy: 0.9600
Epoch 85/200
5/5 [==============================] - 0s 1ms/step - loss: 0.1577 - accuracy: 0.9600
Epoch 86/200
5/5 [==============================] - 0s 997us/step - loss: 0.0529 - accuracy: 1.0000
Epoch 87/200
5/5 [==============================] - 0s 753us/step - loss: 0.0649 - accuracy: 1.0000
Epoch 88/200
5/5 [==============================] - 0s 1ms/step - loss: 0.1318 - accuracy: 0.9600
Epoch 89/200
5/5 [==============================] - 0s 748us/step - loss: 0.1296 - accuracy: 0.9600
Epoch 90/200
5/5 [==============================] - 0s 997us/step - loss: 0.1212 - accuracy: 0.9600
Epoch 91/200
5/5 [==============================] - 0s 992us/step - loss: 0.0724 - accuracy: 1.0000
Epoch 92/200
5/5 [==============================] - 0s 1000us/step - loss: 0.0744 - accuracy: 0.9600
Epoch 93/200
5/5 [==============================] - 0s 998us/step - loss: 0.0509 - accuracy: 1.0000
Epoch 94/200
5/5 [

5/5 [==============================] - 0s 998us/step - loss: 0.0126 - accuracy: 1.0000
Epoch 167/200
5/5 [==============================] - 0s 998us/step - loss: 0.0498 - accuracy: 1.0000
Epoch 168/200
5/5 [==============================] - 0s 748us/step - loss: 0.0522 - accuracy: 1.0000
Epoch 169/200
5/5 [==============================] - 0s 1ms/step - loss: 0.0140 - accuracy: 1.0000
Epoch 170/200
5/5 [==============================] - 0s 991us/step - loss: 0.0375 - accuracy: 1.0000
Epoch 171/200
5/5 [==============================] - 0s 1ms/step - loss: 0.1279 - accuracy: 0.9200
Epoch 172/200
5/5 [==============================] - 0s 997us/step - loss: 0.0308 - accuracy: 1.0000
Epoch 173/200
5/5 [==============================] - 0s 997us/step - loss: 0.0401 - accuracy: 1.0000
Epoch 174/200
5/5 [==============================] - 0s 997us/step - loss: 0.0200 - accuracy: 1.0000
Epoch 175/200
5/5 [==============================] - 0s 992us/step - loss: 0.0433 - accuracy: 1.0000
Epoch 17

In [469]:
def clean_up(sentence):
    sentence_words = nltk.word_tokenize(sentence)
    new_words = []
    for secuence in sentence_words:
        if secuence is list:
            for word in secuense:
                new_words.append(morph.parse(word)[0])
        else:
            new_words.append(morph.parse(secuence)[0])
    sentence_words = [word.normal_form for word in new_words]
    return sentence_words

In [470]:
def bag_of_words(sentence):
    sentence_words = clean_up(sentence)
    bag = [0] * len(words)
    for w in sentence_words:
        for i, word in enumerate(words):
            if word == w:
                bag[i] = 1
    return np.array(bag)

In [471]:
def predict_class(sentense):
    bow=bag_of_words(sentense)
    print(bow)
    result = model.predict(np.array([bow]))[0]
    ERROR = 0.15
    result = [[i, r] for i, r in enumerate(result) if r > ERROR]
    
    result.sort(key=lambda x: x[1], reverse = True)
    return_list = []
    for r in result:
        return_list.append({'intent':clasees[r[0]], 'probability':str(r[1])})
    return return_list

In [472]:
def get_response(intents_list, intents_json):
    tag = intents_list[0]['intent']
    list_of_intents = intents_json['intents']
    for i in list_of_intents:
        if i['tag'] == tag:
            resp = random.choice(i["responses"])
            if resp.find("*") < 0:
                return resp
            else:
                g = eval(resp.split("*")[1])
                return (resp.split("*")[0] + eval(resp.split("*")[1]) )

In [ ]:
print("Demo ChatBot")
while True:
    message = input()
    ints = predict_class(message)
    result = get_response(ints, intents)
    print(result)

Demo ChatBot
првт
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 [==============================] - 0s 35ms/step
Как я могут помочь тебе
ты как
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0]
1/1 [==============================] - 0s 11ms/step
Нормально, сегодня система сбоит
день
[0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
1/1 [==============================] - 0s 13ms/step
Здравствуй, Незнакомец
сегодня
[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0]
1/1 [==============================] - 0s 12ms/step
Сегодня 06/13/22
